In [1]:
#Import relevant libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

### a) El set de datos tiene varias entradas con valores incompletos, para las observaciones que les falta el plazo del monto del préstamo(loan amount term)cambia los NaN por elvalorpromedio de esa columna. Para  las observaciones que  les  falta  indicar  si es  un trabajadorpor  cuenta  propia (self_employed), cambia los NaN por la moda de esa columna.Remueve todas las otras entradas con valores incompletos.

In [2]:
#Import dataset
dataset = pd.read_csv('loan.csv')
#Print number of observations and features
print('This dataset has {} observations with {} features.'.format(dataset.shape[0], dataset.shape[1]))
#Check for null values
dataset.info()

This dataset has 614 observations with 10 features.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Dependents         599 non-null    float64
 1   Education          614 non-null    object 
 2   Self_Employed      582 non-null    object 
 3   ApplicantIncome    614 non-null    int64  
 4   CoapplicantIncome  614 non-null    float64
 5   LoanAmount         592 non-null    float64
 6   Loan_Amount_Term   600 non-null    float64
 7   Credit_History     564 non-null    float64
 8   Property_Area      614 non-null    object 
 9   Loan_Status        614 non-null    object 
dtypes: float64(5), int64(1), object(4)
memory usage: 48.1+ KB


In [3]:
dataset.head()

,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,0.0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,1.0,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,0.0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,0.0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,0.0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [4]:
dataset['Loan_Amount_Term'].fillna(dataset['Loan_Amount_Term'].mean(), inplace=True)

In [5]:
dataset['Self_Employed'].fillna(dataset['Self_Employed'].mode()[0], inplace=True)

In [6]:
dataset = dataset.dropna()

In [7]:
dataset.isnull().sum()

Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

### b) El  set  de  datos  incluye  varias  entradas  de  tipocategóricas (i.e. property_area). Incluye  un comentario  o  una  celda  de  texto (markdown  cell)  explicando  qué  es  one  hot  encodinge impleméntalo entodoslos datos categóricos. Explica cuál hubiera sido el problema de codificar las entradas de property_area de la siguiente forma, Semiurban= 1, Urban = 2, Rural = 3. 

One hot encoding consiste en representar datos categóricos con vectores binarios, a fin de utilizar datos númericos en su lugar. Si se hubiese codificado de la manera descrita en el problema pudiese haber confusión de que los valores mayores que otros implicaran una relación de jerarquía de algún tipo.

In [8]:
dataset['Loan_Status'].replace('Y', 1, inplace=True)
dataset['Loan_Status'].replace('N', 0, inplace=True)

dataset['Self_Employed'].replace('No', 0, inplace=True)
dataset['Self_Employed'].replace('Yes', 1, inplace=True)
dataset.head()

,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,1.0,Graduate,0,4583,1508.0,128.0,360.0,1.0,Rural,0
2,0.0,Graduate,1,3000,0.0,66.0,360.0,1.0,Urban,1
3,0.0,Not Graduate,0,2583,2358.0,120.0,360.0,1.0,Urban,1
4,0.0,Graduate,0,6000,0.0,141.0,360.0,1.0,Urban,1
5,2.0,Graduate,1,5417,4196.0,267.0,360.0,1.0,Urban,1


In [9]:
def one_hot_encode(df):
  for feature_name in df.columns:
    if df[feature_name].dtype == 'object':
      one = pd.get_dummies(df[feature_name])
      df = pd.concat([df,one], axis=1)
      df.drop([feature_name], inplace=True, axis=1)
  return df

In [10]:
dataset = one_hot_encode(dataset)
dataset

,Dependents,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Graduate,Not Graduate,Rural,Semiurban,Urban
1,1.0,0,4583,1508.0,128.0,360.0,1.0,0,1,0,1,0,0
2,0.0,1,3000,0.0,66.0,360.0,1.0,1,1,0,0,0,1
3,0.0,0,2583,2358.0,120.0,360.0,1.0,1,0,1,0,0,1
4,0.0,0,6000,0.0,141.0,360.0,1.0,1,1,0,0,0,1
5,2.0,1,5417,4196.0,267.0,360.0,1.0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,0.0,0,2900,0.0,71.0,360.0,1.0,1,1,0,1,0,0
610,3.0,0,4106,0.0,40.0,180.0,1.0,1,1,0,1,0,0
611,1.0,0,8072,240.0,253.0,360.0,1.0,1,1,0,0,0,1
612,2.0,0,7583,0.0,187.0,360.0,1.0,1,1,0,0,0,1


### c) ¿Qué   métrica   utilizarías   para   determinar   el   rendimiento   del   modelo,   la precisióno   la exhaustividad? Justifica tu respuesta.

Precisión pues responde a la pregunta de "¿Qué proporción de identificaciones positivas fue correcta?" y queremos medir los falsos positivos

### d) Divide  el  set  de  datos  con  una  distribución  de  80%  para  el  entrenamiento  y  20%  de  testing.Experimenta  con  diferentes  arquitecturas(varía  la  cantidad  de  capas,  neuronas,  funciones de activación, tasa de aprendizaje, etc)y reporta el mejor rendimiento que obtuviste.

Utilicé la eramienta de keras tuner para probar diferentes arquitecturas y elegir el mejor modelo de ellos.

In [11]:
#Separate input data and labels
X = dataset.iloc[:,0:12] 
y = dataset.iloc[:, 7].values

#Split data into training and testing 
 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [12]:
from keras import backend as K

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [15]:
import keras
from keras import Sequential
from keras.layers import Dense
from keras import optimizers
import keras_tuner as kt
from tensorflow import keras

def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Dense(
      hp.Choice('units', [16, 32, 64]),
      activation='relu'))
    model.add(keras.layers.Dense(
      hp.Choice('units', [16, 32, 64]),
      activation='softmax'))
    model.add(keras.layers.Dense(1, activation='linear'))
    sgd = optimizers.SGD(
        hp.Choice('learning_rate', [0.01, 0.02, 0.001]))
    model.compile(optimizer="adam", loss='binary_crossentropy',  metrics=[precision_m])
    return model

In [16]:
tuner = kt.RandomSearch(
    build_model,
    objective="val_loss",
    max_trials=10)

In [17]:
tuner.search(X_train,y_train,batch_size=32, validation_split = 0.2, epochs=30)

Trial 8 Complete [00h 00m 03s]
val_loss: 0.6554303169250488

Best val_loss So Far: 0.6488664150238037
Total elapsed time: 00h 00m 24s
INFO:tensorflow:Oracle triggered exit


In [18]:
best_model = tuner.get_best_models()[0]

Mejor rendimiento:

In [19]:
best_model.fit(X_train, y_train, batch_size=10, epochs=30, shuffle=True, verbose=2)

Epoch 1/30
43/43 - 1s - loss: 0.6302 - precision_m: 0.6942
Epoch 2/30
43/43 - 0s - loss: 0.6298 - precision_m: 0.6932
Epoch 3/30
43/43 - 0s - loss: 0.6352 - precision_m: 0.6901
Epoch 4/30
43/43 - 0s - loss: 0.6208 - precision_m: 0.6953
Epoch 5/30
43/43 - 0s - loss: 0.6188 - precision_m: 0.6919
Epoch 6/30
43/43 - 0s - loss: 0.6179 - precision_m: 0.6849
Epoch 7/30
43/43 - 0s - loss: 0.6171 - precision_m: 0.6884
Epoch 8/30
43/43 - 0s - loss: 0.6167 - precision_m: 0.6953
Epoch 9/30
43/43 - 0s - loss: 0.6164 - precision_m: 0.6849
Epoch 10/30
43/43 - 0s - loss: 0.6163 - precision_m: 0.6919
Epoch 11/30
43/43 - 0s - loss: 0.6161 - precision_m: 0.6919
Epoch 12/30
43/43 - 0s - loss: 0.6161 - precision_m: 0.6884
Epoch 13/30
43/43 - 0s - loss: 0.6158 - precision_m: 0.6884
Epoch 14/30
43/43 - 0s - loss: 0.6158 - precision_m: 0.6953
Epoch 15/30
43/43 - 0s - loss: 0.6156 - precision_m: 0.6884
Epoch 16/30
43/43 - 0s - loss: 0.6159 - precision_m: 0.6953
Epoch 17/30
43/43 - 0s - loss: 0.6157 - precision

### e) Entrena en Google colab por 500 épocas el mejor modelo obtenido. Reporta cuánto tiempo tardó en CPU vs GPU vs TPU.

**CPU**


CPU times: user 32.9 s, sys: 3.21 s, total: 36.1 s
Wall time: 29.9 s

In [23]:
%%time
best_model.fit(X_train, y_train, batch_size=10, epochs=500, shuffle=True, verbose=2)

Epoch 1/500
43/43 - 0s - loss: 0.6279 - precision_m: 0.6767
Epoch 2/500
43/43 - 0s - loss: 0.6280 - precision_m: 0.6767
Epoch 3/500
43/43 - 0s - loss: 0.6278 - precision_m: 0.6767
Epoch 4/500
43/43 - 0s - loss: 0.6279 - precision_m: 0.6802
Epoch 5/500
43/43 - 0s - loss: 0.6285 - precision_m: 0.6837
Epoch 6/500
43/43 - 0s - loss: 0.6274 - precision_m: 0.6733
Epoch 7/500
43/43 - 0s - loss: 0.6277 - precision_m: 0.6802
Epoch 8/500
43/43 - 0s - loss: 0.6277 - precision_m: 0.6767
Epoch 9/500
43/43 - 0s - loss: 0.6278 - precision_m: 0.6837
Epoch 10/500
43/43 - 0s - loss: 0.6276 - precision_m: 0.6733
Epoch 11/500
43/43 - 0s - loss: 0.6278 - precision_m: 0.6767
Epoch 12/500
43/43 - 0s - loss: 0.6281 - precision_m: 0.6802
Epoch 13/500
43/43 - 0s - loss: 0.6276 - precision_m: 0.6837
Epoch 14/500
43/43 - 0s - loss: 0.6278 - precision_m: 0.6837
Epoch 15/500
43/43 - 0s - loss: 0.6279 - precision_m: 0.6802
Epoch 16/500
43/43 - 0s - loss: 0.6276 - precision_m: 0.6698
Epoch 17/500
43/43 - 0s - loss: 0

**GPU**

CPU times: user 52.2 s, sys: 4.94 s, total: 57.2 s
Wall time: 1min 21s

In [23]:
%%time
best_model.fit(X_train, y_train, batch_size=10, epochs=500, shuffle=True, verbose=2)

Epoch 1/500
43/43 - 0s - loss: 0.6004 - precision_m: 0.7128
Epoch 2/500
43/43 - 0s - loss: 0.6009 - precision_m: 0.7128
Epoch 3/500
43/43 - 0s - loss: 0.6005 - precision_m: 0.7163
Epoch 4/500
43/43 - 0s - loss: 0.6003 - precision_m: 0.7163
Epoch 5/500
43/43 - 0s - loss: 0.6006 - precision_m: 0.7163
Epoch 6/500
43/43 - 0s - loss: 0.6012 - precision_m: 0.7128
Epoch 7/500
43/43 - 0s - loss: 0.6004 - precision_m: 0.7128
Epoch 8/500
43/43 - 0s - loss: 0.6010 - precision_m: 0.7093
Epoch 9/500
43/43 - 0s - loss: 0.6006 - precision_m: 0.7163
Epoch 10/500
43/43 - 0s - loss: 0.6004 - precision_m: 0.7128
Epoch 11/500
43/43 - 0s - loss: 0.6009 - precision_m: 0.7163
Epoch 12/500
43/43 - 0s - loss: 0.6007 - precision_m: 0.7128
Epoch 13/500
43/43 - 0s - loss: 0.6006 - precision_m: 0.7128
Epoch 14/500
43/43 - 0s - loss: 0.6005 - precision_m: 0.7128
Epoch 15/500
43/43 - 0s - loss: 0.6007 - precision_m: 0.7163
Epoch 16/500
43/43 - 0s - loss: 0.6004 - precision_m: 0.7093
Epoch 17/500
43/43 - 0s - loss: 0

**TPU**

CPU times: user 29.3 s, sys: 2 s, total: 31.3 s
Wall time: 41 s

In [20]:
%%time
best_model.fit(X_train, y_train, batch_size=10, epochs=500, shuffle=True, verbose=2)

Epoch 1/500
43/43 - 0s - loss: 0.6153 - precision_m: 0.6953
Epoch 2/500
43/43 - 0s - loss: 0.6158 - precision_m: 0.6919
Epoch 3/500
43/43 - 0s - loss: 0.6155 - precision_m: 0.6884
Epoch 4/500
43/43 - 0s - loss: 0.6157 - precision_m: 0.6919
Epoch 5/500
43/43 - 0s - loss: 0.6157 - precision_m: 0.6884
Epoch 6/500
43/43 - 0s - loss: 0.6160 - precision_m: 0.6919
Epoch 7/500
43/43 - 0s - loss: 0.6154 - precision_m: 0.6919
Epoch 8/500
43/43 - 0s - loss: 0.6155 - precision_m: 0.6919
Epoch 9/500
43/43 - 0s - loss: 0.6156 - precision_m: 0.6919
Epoch 10/500
43/43 - 0s - loss: 0.6154 - precision_m: 0.6919
Epoch 11/500
43/43 - 0s - loss: 0.6154 - precision_m: 0.6919
Epoch 12/500
43/43 - 0s - loss: 0.6156 - precision_m: 0.6953
Epoch 13/500
43/43 - 0s - loss: 0.6154 - precision_m: 0.6849
Epoch 14/500
43/43 - 0s - loss: 0.6155 - precision_m: 0.6919
Epoch 15/500
43/43 - 0s - loss: 0.6160 - precision_m: 0.6919
Epoch 16/500
43/43 - 0s - loss: 0.6156 - precision_m: 0.6919
Epoch 17/500
43/43 - 0s - loss: 0